In [ ]:
import numpy as np
import torch
from scipy.io import loadmat
from ml import BrainDataset, PolicyEstimator, reinforce, sample_batch_fn
from utils import train_cv_test_split
from cust_plot import plot

In [ ]:
# Input Parameters
num_fns = 9
res = 68
subj = np.arange(484)
epochs = 1
batch = 4
sample = 100
hidden_units = 10
lr = 0.001
save_path = None
load_path = None
save_freq = 1
log = True

In [ ]:
# Read brain data (change file locations as necessary)
sc = loadmat(f'data/subjfiles_SC{res}.mat')
fc = loadmat(f'data/subjfiles_FC{res}.mat')
sc = np.array([sc[f's{str(z+1).zfill(3)}'] for z in subj])
fc = np.array([fc[f's{str(z+1).zfill(3)}'] for z in subj])
euc_dist = loadmat('data/euc_dist.mat')[f'eu{res}']
hubs = np.loadtxt(f'data/hubs_{res}.txt', dtype=np.int, delimiter=',')
regions = np.loadtxt(f'data/regions_{res}.txt', dtype=np.int, delimiter=',')

In [ ]:
# Init network parameters
pe = PolicyEstimator(res, num_fns, hidden_units=hidden_units)
opt = torch.optim.Adam(pe.network.parameters(), lr=lr)

In [ ]:
# Init new/load previous training data
if load_path:
    # Load from checkpoint
    plt_data = torch.load(load_path)
    pe.network.load_state_dict(plt_data.pop('model_state_dict'))
    opt.load_state_dict(plt_data.pop('optimizer_state_dict'))
else:
    # New
    plt_data = {
        'epochs': 0,
        'epoch_seconds': [],
        'rewards': [],
        'success': [],
        'mu': [[] for _ in range(num_fns)],
        'sig': [[] for _ in range(num_fns)]}
    plt_data['train_idx'], plt_data['cv_idx'], plt_data['test_idx'] = train_cv_test_split(subj, train_pct=0.6, cv_pct=0.2)

In [ ]:
# Train / cv / test split
train_idx, cv_idx, test_idx = plt_data['train_idx'], plt_data['cv_idx'], plt_data['test_idx']
train_data = BrainDataset(sc[train_idx], fc[train_idx], euc_dist, hubs, regions)
cv_data = BrainDataset(sc[cv_idx], fc[cv_idx], euc_dist, hubs, regions)
test_data =  BrainDataset(sc[test_idx],  fc[test_idx],  euc_dist, hubs, regions)

In [ ]:
# Train network
reinforce(pe, opt, train_data, epochs=10, batch=batch, sample=sample, lr=lr, plt_data=plt_data, save_path=save_path, save_freq=save_freq, log=log)

In [ ]:
# Visualise
plot(plt_data=plt_data, num_fns=num_fns, plt_avg=50)

In [ ]:
# Evaluate
fn_weights = [mu[-1] for mu in plt_data['mu']]
test_rewards = np.zeros(len(test_data))
test_success = np.zeros(len(test_data))
for i, (_, sp, pp, sample_idx) in enumerate(test_data):
    pp.fn_weights = fn_weights
    test_rewards[i], test_success[i] = sample_batch_fn(pp, sp, sample, sample_idx)

test_rewards = test_rewards.mean()
test_success = test_success.mean()
print(test_rewards, test_success)